<a href="https://colab.research.google.com/github/nspiegeln/iml_2022/blob/main/inceptionv3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#load data
from google.colab import files
uploaded = files.upload()

Saving test_triplets.txt to test_triplets.txt
Saving train_triplets.txt to train_triplets.txt


In [3]:
#load food.zip
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
!unzip gdrive/My\ Drive/food/food.zip

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
  inflating: food/05000.jpg          
  inflating: food/05001.jpg          
  inflating: food/05002.jpg          
  inflating: food/05003.jpg          
  inflating: food/05004.jpg          
  inflating: food/05005.jpg          
  inflating: food/05006.jpg          
  inflating: food/05007.jpg          
  inflating: food/05008.jpg          
  inflating: food/05009.jpg          
  inflating: food/05010.jpg          
  inflating: food/05011.jpg          
  inflating: food/05012.jpg          
  inflating: food/05013.jpg          
  inflating: food/05014.jpg          
  inflating: food/05015.jpg          
  inflating: food/05016.jpg          
  inflating: food/05017.jpg          
  inflating: food/05018.jpg          
  inflating: food/05019.jpg          
  inflating: food/05020.jpg          
  inflating: food/05021.jpg          
  inflating: food/05022.jpg          
  inflating: food/05023.jpg          
  inflating: food/050

In [2]:
import torch
import numpy as np
from matplotlib import image
import matplotlib.pyplot as plt
from torch import nn
from torch import optim
import tensorflow as tf
from tensorflow import keras
import torch.nn.functional as F
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input, Lambda
from torchvision import datasets, transforms, models
from sklearn.model_selection import train_test_split
from PIL import Image
#from skimage.transform import resize
from torch.utils.data import Dataset, DataLoader
import os
import cv2

In [5]:
# with open ('train_triplets.txt') as f:
#     train_id = f.readline()
#     f.close()

# with open ('test_triplets.txt') as f:
#     test_id = f.readlines()
#     f.close()

def transform_img(image):
  img_shape = 299
  res = np.zeros((img_shape,img_shape,3))
  res[:,:,0] = cv2.resize(image[:,:,0], dsize=(img_shape, img_shape), interpolation=cv2.INTER_CUBIC)
  res[:,:,1] = cv2.resize(image[:,:,1], dsize=(img_shape, img_shape), interpolation=cv2.INTER_CUBIC)
  res[:,:,2] = cv2.resize(image[:,:,2], dsize=(img_shape, img_shape), interpolation=cv2.INTER_CUBIC)
  return res


train_transforms = transforms.Compose([transforms.Resize((256,256)),
                                       transforms.RandomCrop(224),
                                       transforms.RandomHorizontalFlip(p=0.5),
                                       transforms.RandomVerticalFlip(p=0.5),
                                       transforms.RandomRotation(degrees = 90)])

test_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224)])

In [6]:
def get_default_device():
    """Picking GPU if available or else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

device = get_default_device()

In [7]:
inception = tf.keras.applications.InceptionV3(include_top=False, weights="imagenet", pooling='avg')
inception.trainable = False

#define input
input_shape = (299,299,3)
x = x_in = Input(shape=input_shape)
x = inception(x)

model = Model(inputs=x_in, outputs=x)

In [4]:
class TripletImageDataset(Dataset):
    def __init__(self, image_folder_name, triplets_file_name, transforms):
        self.image_dir =  image_folder_name
        self.triplets_file_name = triplets_file_name
        self.transforms = transforms
        triplets = []
        for line in open(triplets_file_name):
            ids = line.split()
            triplets.append([ids[0], ids[1], ids[2]])
            break
        self.triplets = triplets

    def __getitem__(self, index):
        triplet_ids = self.triplets[index]
        path_1 = os.path.join(self.image_dir, triplet_ids[0]+'.jpg')
        path_2 = os.path.join(self.image_dir, triplet_ids[1]+'.jpg')
        path_3 = os.path.join(self.image_dir, triplet_ids[2]+'.jpg')

        img_1 = transform_img(plt.imread(path_1))
        img_2 = transform_img(plt.imread(path_2))
        img_3 = transform_img(plt.imread(path_3))

        img_1 = tf.keras.applications.inception_v3.preprocess_input(img_1)
        img_2 = tf.keras.applications.inception_v3.preprocess_input(img_2)
        img_3 = tf.keras.applications.inception_v3.preprocess_input(img_3)

        return img_1, img_2, img_3

    def __len__(self):
        return len(self.triplets) 

batch_size=1
train_dataset = TripletImageDataset('food', 'train_triplets.txt', train_transforms)
test_dataset = TripletImageDataset('food', 'test_triplets.txt', test_transforms)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [50]:

for i, batch in enumerate(train_loader):
  triplet = np.concatenate((batch[0], batch[1], batch[2]), axis = 0)
  length = triplet.shape[5]
  
  print(length)
  break

IndexError: ignored

In [46]:
def get_train_tensor(train_loader):
  train_tensor = []
  labels = []
  
  for i, batch in enumerate(train_loader):
    length = batch[0].shape[0]
    triplet = np.concatenate((batch[0], batch[1], batch[2]), axis = 0)
    reverse_triplet = np.concatenate((batch[0], batch[2], batch[1]), axis = 0)

    train_tensor.append(triplet)

    labels.append(np.ones((length,1)))

    train_tensor.append(reverse_triplet)
    labels.append(np.zeros((length, 1)))

  train_tensor = np.array(train_tensor)
  labels = np.array(labels)
  return train_tensor, labels

def get_test_tensor(test_loader):
  test_tensor = []
  for i, batch in enumerate(test_loader):
    triplet = np.concatenate((batch[0], batch[1], batch[2]), axis = -1)
    test_tensor.append(triplet)

  test_tensor = np.array(test_tensor)
  return test_tensor

In [18]:
train_tensors, labels = get_train_tensor(train_loader)
print(len(train_tensors))

2


In [47]:
train_tensors, labels = get_train_tensor(train_loader)
#train_tensors.to(device), labels.to(device)

x = x_in = Input(train_tensors.shape[1:])
x = Activation('relu')(x)
x = Dropout(0.7)(x)
x = Dense(2048)(x)
x = Activation('relu')(x)
x = Dense(512)(x)
x = Activation('relu')(x)
x = Dense(128)(x)
x = Activation('relu')(x)
x = Dense(32)(x)
x = Activation('relu')(x)
x = Dense(8)(x)
x = Activation('relu')(x)
x = Dense(1)(x)
x = Activation('sigmoid')(x)
model = Model(inputs=x_in, outputs=x)

#model.to(device)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print("Training")
model.fit(x = train_tensors, y = labels, epochs=1)
print("Training completed!")

print("Predicting")
test_tensors = get_train_tensor(test_loader)
test_tensors.to(device)

y_test = model.predict(test_tensors)
print("Predicting completed!")


Training


ValueError: ignored